### what do we want in a box score?

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from pdb import set_trace
import glob
from collections import Counter

In [16]:
ls

README.md  figs/      library/   notebooks/


cd 'blueprint-stats/'

### COMBINE GAMExGAME POSSESSION DATA INTO ONE POSS_DF

In [17]:
#give file path abbreviation
poss_path_abbr = 'library/possessions/Possessions vs.*.csv'
#create list of files that match the possession convention
file_list = glob.glob(poss_path_abbr)
#create an empty df
poss_df = pd.DataFrame()
#loop through each file in the list
for file in file_list:
    #add an oppnent depending on the name of the file
    opponent = file.split('vs. ')[1].split(' 2019')[0]
    df_ = pd.read_csv(file)
    #add oppenent name column
    df_['opponent'] = opponent
    #append file to the overall poss_df
    poss_df = poss_df.append(df_)

In [18]:
poss_df.head()

,Created,Point,Possession,Started point on offense?,Scored?,Start X (0 -> 1 = left sideline -> right sideline),Start Y (0 -> 1 = back of opponent endzone -> back of own endzone),Initiator,Passes,Secondary assist,Assist,Goal,Thrower error,Receiver error,Stalled out,opponent
0,2020-02-26 20:24:33,1,1,0,0,0.187023,0.822740,Andrew Edelman,4,NaN,NaN,NaN,Andrew Edelman,NaN,NaN,HIP
1,2020-02-26 20:26:03,2,1,1,1,0.704198,0.802260,Victor Kao,5,Ian McInerney,Jeff Voss,Gabe Stump,NaN,NaN,NaN,HIP
2,2020-02-26 20:27:57,4,1,1,1,0.498092,0.786017,Owen Marschall,7,Sam Pollack,Odie,Jeff Voss,NaN,NaN,NaN,HIP
3,2020-02-26 20:30:04,6,1,1,0,0.492366,0.822740,Owen Marschall,3,NaN,NaN,NaN,Gabe Stump,NaN,NaN,HIP
4,2020-02-26 20:32:02,7,1,0,1,0.251908,0.807203,Zero,2,Zero,Odie,Sam Alston,NaN,NaN,NaN,HIP


### COMBINE GAMExGAME PASSES DATA INTO ONE PASSES_DF

In [19]:
#give file path abbreviation
passes_path_abbr = 'library/passes/Passes vs.*.csv'
#create list of files that match the passes convention
file_list = glob.glob(passes_path_abbr)
#create an empty df
passes_df = pd.DataFrame()
#loop through each file in the list
for file in file_list:
    #add an oppnent depending on the name of the file
    opponent = file.split('vs. ')[1].split(' 2019')[0]
    df_ = pd.read_csv(file)
    #add oppenent name column
    df_['opponent'] = opponent
    #append file to the overall passes_df
    passes_df = passes_df.append(df_)

#remove all of the empty data with no thrower nor reciever
passes_df = passes_df.dropna(subset=['Thrower', 'Receiver'])

In [20]:
passes_df.head()

,Created,Point,Possession,Thrower,Receiver,Turnover?,Thrower error?,Receiver error?,Throw to endzone?,Assist?,...,From sideline?,To sideline?,Distance (m),Forward distance (m),Left-to-right distance (m),Start X (0 -> 1 = left sideline -> right sideline),Start Y (0 -> 1 = back of opponent endzone -> back of own endzone),End X (0 -> 1 = left sideline -> right sideline),End Y (0 -> 1 = back of opponent endzone -> back of own endzone),opponent
0,2020-02-26 22:56:29,1,1,Evan Ma,Max Cohen,0,0,0,0,0,...,0,0,11.115858,4.943502,-9.956108,0.265267,0.816384,0.534351,0.766949,Lost Boys
1,2020-02-26 22:56:39,1,1,Max Cohen,Victor Kao,0,0,0,0,0,...,0,0,5.945641,1.624298,5.719467,0.534351,0.766949,0.379771,0.750706,Lost Boys
2,2020-02-26 22:56:46,1,1,Victor Kao,Carter Thallon,0,0,0,0,0,...,0,0,14.086006,6.497174,-12.498092,0.379771,0.750706,0.717557,0.685734,Lost Boys
3,2020-02-26 22:56:49,1,1,Carter Thallon,Odie,0,0,0,0,0,...,0,0,8.952368,4.802257,7.555344,0.717557,0.685734,0.513359,0.637712,Lost Boys
4,2020-02-26 22:56:55,1,1,Odie,Carter Thallon,0,0,0,0,0,...,0,0,7.909012,-3.954798,6.849237,0.513359,0.637712,0.328244,0.677260,Lost Boys


### MERGE THE PASSES_DF WITH THE POSS_DF

In [21]:
full_poss_df = pd.merge(passes_df,poss_df,how='left',on=['Point','Possession','opponent'],copy=False)
full_poss_df = full_poss_df.reset_index(drop=True)
full_poss_df = full_poss_df.drop(columns=['Created_y','Start X (0 -> 1 = left sideline -> right sideline)_y',
                  'Start Y (0 -> 1 = back of opponent endzone -> back of own endzone)_y'])
full_poss_df.to_csv('library/possessions/BP_2019_Full_Possession_Data.csv')

In [22]:
full_poss_df.columns = [x.replace('_x','') for x in full_poss_df.columns]

In [23]:
full_poss_df.columns

Index(['Created', 'Point', 'Possession', 'Thrower', 'Receiver', 'Turnover?',
       'Thrower error?', 'Receiver error?', 'Throw to endzone?', 'Assist?',
       'Secondary assist?', 'Huck?', 'Swing?', 'Dump?', 'From sideline?',
       'To sideline?', 'Distance (m)', 'Forward distance (m)',
       'Left-to-right distance (m)',
       'Start X (0 -> 1 = left sideline -> right sideline)',
       'Start Y (0 -> 1 = back of opponent endzone -> back of own endzone)',
       'End X (0 -> 1 = left sideline -> right sideline)',
       'End Y (0 -> 1 = back of opponent endzone -> back of own endzone)',
       'opponent', 'Started point on offense?', 'Scored?', 'Initiator',
       'Passes', 'Secondary assist', 'Assist', 'Goal', 'Thrower error',
       'Receiver error', 'Stalled out'],
      dtype='object')

In [24]:
#create a unique posession index based on opponent, point#, possession#
for x in full_poss_df.index:
    full_poss_df.loc[x,'possession_index'] = full_poss_df.loc[x,'opponent'] + ' | ' + \
    str(full_poss_df.loc[x,'Point']) + '.' + \
    str(full_poss_df.loc[x,'Possession'])

### ADD LINEUPS FOR EACH POINT

In [25]:
#create a lineup dataframe with subset of full poss columns
lineup_df = full_poss_df[['possession_index','opponent','Point','Possession']]
lineup_df

,possession_index,opponent,Point,Possession
0,Lost Boys | 1.1,Lost Boys,1,1
1,Lost Boys | 1.1,Lost Boys,1,1
2,Lost Boys | 1.1,Lost Boys,1,1
3,Lost Boys | 1.1,Lost Boys,1,1
4,Lost Boys | 1.1,Lost Boys,1,1
...,...,...,...,...
1158,Lantern | 14.1,Lantern,14,1
1159,Lantern | 14.1,Lantern,14,1
1160,Lantern | 14.1,Lantern,14,1
1161,Lantern | 14.1,Lantern,14,1


In [26]:
#create opp_pt column
for x in lineup_df.index:
    lineup_df.loc[x,'opp_pt'] = lineup_df.loc[x,'opponent'] + '.' + str(lineup_df.loc[x,'Point'])

/Users/maxcohen/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/maxcohen/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [27]:
#add lineups as dictionaries for each point in all games

In [28]:
lantern_lineup_dict = {'opp_pt':['Lantern.1','Lantern.2','Lantern.3','Lantern.4','Lantern.5','Lantern.6','Lantern.7',
                                'Lantern.8','Lantern.9','Lantern.10','Lantern.11','Lantern.12','Lantern.13','Lantern.14'],
                      'lineup':[[],
                                ['Max Cohen','Jesse Gan','Owen Marschall','Odie','Sam Pollack','Jeff Voss','Geo Zheng'],
                               ['Alon Brown','Andrew Edelman','Jon Kwan','Evan Ma','Ian McInerney','Stouf','Towell'],
                               ['Max Cohen','Owen Marschall','Odie','Sam Pollack','Gabe Stump','Carter Thallon'],
                               ['Alon Brown','Andrew Edelman','Sam Feder','Jesse Gan','Josh Imhoff','Towell','Carter Wang'],
                               ['Alon Brown','Andrew Edelman','Jon Kwan','Evan Ma','Swan','Ben West','Zero'],
                               ['Alon Brown','Jesse Gan','Jon Kwan','Evan Ma','Ian McInerney','Stouf','Towell'],
                               ['Max Cohen','Owen Marschall','Odie','Gabe Stump','Carter Thallon','Jeff Voss','Geo Zheng'],
                               ['Andrew Edelman','Sam Feder','Jesse Gan','Josh Imhoff','Evan Ma','Towell','Carter Wang'],
                               ['Max Cohen','Owen Marschall','Sam Pollack','Gabe Stump','Carter Thallon','Jeff Voss','Geo Zheng'],
                               ['Alon Brown','Andrew Edelman','Sam Feder','Jesse Gan','Ian McInerney','Towell','Carter Wang'],
                               ['Max Cohen','Owen Marschall','Odie','Sam Pollack','Gabe Stump','Jeff Voss','Geo Zheng'],
                               ['Alon Brown','Andrew Edelman','Sam Feder','Jesse Gan','Ian McInerney','Stouf','Towell'],
                               ['Max Cohen','Owen Marschall','Odie','Sam Pollack','Carter Thallon','Jeff Voss','Geo Zheng']]}


In [29]:
wrench_lineup_dict = {'opp_pt':['Big Wrench.1','Big Wrench.2','Big Wrench.3','Big Wrench.4','Big Wrench.5',
                                'Big Wrench.6','Big Wrench.7','Big Wrench.8','Big Wrench.9','Big Wrench.10',
                                'Big Wrench.11','Big Wrench.12','Big Wrench.13','Big Wrench.14','Big Wrench.15',
                               'Big Wrench.16','Big Wrench.17','Big Wrench.18','Big Wrench.19','Big Wrench.20'],
                      'lineup':[['Max Cohen','Andrew Edelman','Sam Feder','Owen Marschall','Odie','Gabe Stump','Towell'],
                               ['Alon Brown','Jon Kwan','Ian McInerney','Stouf','Towell','Ruozhou Ye','Geo Zheng'],
                               ['Andrew Edelman','Sam Feder','Jesse Gan','Josh Imhoff','Evan Ma','Carter Wang','Ruozhou Ye'],
                               ['Alon Brown','Jesse Gan','Jon Kwan','Swan','Ben West','Ruozhou Ye','Zero'],
                               ['Max Cohen','Owen Marschall','Odie','Gabe Stump','Carter Thallon','Jeff Voss','Geo Zheng'],
                               ['Andrew Edelman','Sam Feder','Jesse Gan','Josh Imhoff','Evan Ma','Carter Wang','Ruozhou Ye'],
                               ['Andrew Edelman','Sam Feder','Jesse Gan','Jon Kwan','Swan','Ben West','Ruozhou Ye'],
                               ['Max Cohen','Owen Marschall','Odie','Sam Pollack','Carter Thallon','Jeff Voss','Geo Zheng'],
                               ['Alon Brown','Andrew Edelman','Jon Kwan','Ian McInerney','Stouf','Towell','Ruozhou Ye'],
                               ['Josh Imhoff','Evan Ma','Swan','Ben West','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Max Cohen','Owen Marschall','Odie','Sam Pollack','Carter Thallon','Jeff Voss','Geo Zheng'],
                               ['Alon Brown','Andrew Edelman','Sam Feder','Jesse Gan','Ian McInerney','Towell','Carter Wang'],
                               ['Andrew Edelman','Jesse Gan','Josh Imhoff','Evan Ma','Swan','Ben West','Zero'],
                               ['Alon Brown','Sam Feder','Jesse Gan','Evan Ma','Ian McInerney','Towell','Carter Wang'],
                               ['Max Cohen','Owen Marschall','Odie','Sam Pollack','Gabe Stump','Jeff Voss','Geo Zheng'],
                               ['Alon Brown','Andrew Edelman','Jon Kwan','Evan Ma','Ian McInerney','Stouf','Towell'],
                               ['Max Cohen','Owen Marschall','Odie','Sam Pollack','Gabe Stump','Carter Thallon','Geo Zheng'],
                               ['Alon Brown','Andrew Edelman','Jesse Gan','Jon Kwan','Swan','Ben West','Zero'],
                               ['Max Cohen','Owen Marschall','Sam Pollack','Gabe Stump','Carter Thallon','Jeff Voss','Geo Zheng'],
                               ['Max Cohen','Owen Marschall','Odie','Sam Pollack','Carter Thallon','Jeff Voss','Geo Zheng']]}

In [30]:
circus_lineup_dict = {'opp_pt':['Red Circus.1','Red Circus.2','Red Circus.3','Red Circus.4','Red Circus.5',
                                'Red Circus.6','Red Circus.7','Red Circus.8','Red Circus.9','Red Circus.10',
                                'Red Circus.11','Red Circus.12','Red Circus.13','Red Circus.14','Red Circus.15',
                               'Red Circus.16','Red Circus.17','Red Circus.18','Red Circus.19','Red Circus.20'],
                      'lineup':[['Andrew Edelman','Sam Feder','Ian McInerney','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Jeff Voss'],
                               ['Alon Brown','Andrew Edelman','Jesse Gan','Jon Kwan','Swan','Ben West','Zero'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Sam Pollack','Gabe Stump','Carter Thallon','Geo Zheng'],
                               ['Andrew Edelman','Sam Feder','Jesse Gan','Josh Imhoff','Evan Ma','Carter Wang','Ruozhou Ye'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Jeff Voss'],
                               ['Alon Brown','Jon Kwan','Ian McInerney','Juleon Robinson','Stouf','Towell','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Sam Pollack','Gabe Stump','Carter Thallon'],
                               ['Andrew Edelman','Sam Feder','Ian McInerney','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                               ['Eliot Bemis','Andrew Edelman','Sam Feder','Jesse Gan','Josh Imhoff','Stouf','Ruozhou Ye'],
                               ['Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Gabe Stump','Jeff Voss'],
                               ['Andrew Edelman','Ian McInerney','Odie','Juleon Robinson','Towell','Carter Wang','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Sam Pollack','Gabe Stump','Carter Thallon'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Jeff Voss'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Sam Pollack','Gabe Stump','Carter Thallon','Geo Zheng'],
                               ['Alon Brown','Andrew Edelman','Jesse Gan','Jon Kwan','Swan','Ben West','Zero'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Jeff Voss'],
                               ['Alon Brown','Andrew Edelman','Sam Feder','Jesse Gan','Towell','Carter Wang','Ruozhou Ye'],
                               ['Andrew Edelman','Josh Imhoff','Ian McInerney','Odie','Juleon Robinson','Jeff Voss','Ruozhou Ye'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Sam Pollack','Gabe Stump','Carter Thallon','Geo Zheng']]}

In [31]:
shade_lineup_dict = {'opp_pt':['Shade.1','Shade.2','Shade.3','Shade.4','Shade.5',
                                'Shade.6','Shade.7','Shade.8','Shade.9','Shade.10',
                                'Shade.11','Shade.12','Shade.13','Shade.14','Shade.15',
                               'Shade.16','Shade.17','Shade.18','Shade.19','Shade.20',
                              'Shade.21','Shade.22','Shade.23','Shade.24'],
                      'lineup':[['Sam Alston','Andrew Edelman','Sam Feder','Jesse Gan','Towell','Carter Wang','Ruozhou Ye'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Sam Feder','Ian McInerney','Towell','Carter Wang','Ruozhou Ye'],
                               ['Alon Brown','Jesse Gan','Jon Kwan','Evan Ma','Swan','Towell','Ruozhou Ye'],
                               ['Sam Alston','Eliot Bemis','Alon Brown','Andrew Edelman','Stouf','Carter Wang','Ruozhou Ye'],
                               ['Andrew Edelman','Sam Feder','Evan Ma','Ian McInerney','Swan','Ruozhou Ye','Zero'],
                               ['Alon Brown','Jesse Gan','Victor Kao','Jon Kwan','Stouf','Carter Wang','Geo Zheng'],
                               ['Max Cohen','Victor Kao','Sam Pollack','Gabe Stump','Carter Thallon','Jeff Voss','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Sam Feder','Jesse Gan','Towell','Carter Wang','Ruozhou Ye'],
                               ['Alon Brown','Evan Ma','Ian McInerney','Odie','Stouf','Swan','Geo Zheng'],
                               ['Max Cohen','Victor Kao','Owen Marschall','Sam Pollack','Gabe Stump','Carter Thallon','Jeff Voss'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Jeff Voss'],
                                ['Sam Alston','Andrew Edelman','Sam Feder','Ian McInerney','Towell','Carter Wang','Ruozhou Ye'],
                               ['Max Cohen','Victor Kao','Sam Pollack','Gabe Stump','Carter Thallon','Jeff Voss','Geo Zheng'],
                               ['Sam Alston','Eliot Bemis','Max Cohen','Victor Kao','Sam Pollack','Jeff Voss','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Odie','Sam Pollack','Jeff Voss','Ruozhou Ye','Geo Zheng'],
                               ['Alon Brown','Andrew Edelman','Sam Feder','Jesse Gan','Carter Wang','Ruozhou Ye','Zero'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Jeff Voss'],
                               ['Eliot Bemis','Andrew Edelman','Sam Feder','Victor Kao','Odie','Towell','Geo Zheng'],
                               ['Alon Brown','Andrew Edelman','Sam Feder','Swan','Towell','Carter Wang','Ruozhou Ye'],
                               ['Andrew Edelman','Sam Feder','Jon Kwan','Ian McInerney','Stouf','Ruozhou Ye','Geo Zheng'],
                               ['Alon Brown','Jesse Gan','Evan Ma','Swan','Carter Wang','Ruozhou Ye','Zero'],
                               ['Alon Brown','Andrew Edelman','Jon Kwan','Odie','Stouf','Towell','Geo Zheng']]}

In [32]:
vault_lineup_dict = {'opp_pt':['Vault.1','Vault.2','Vault.3','Vault.4','Vault.5',
                                'Vault.6','Vault.7','Vault.8','Vault.9','Vault.10',
                                'Vault.11','Vault.12','Vault.13','Vault.14','Vault.15',
                               'Vault.16','Vault.17','Vault.18','Vault.19','Vault.20',
                              'Vault.21','Vault.22'],
                      'lineup':[['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                               ['Max Cohen','Victor Kao','Evan Ma','Sam Pollack','Gabe Stump','Carter Thallon','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye','Geo Zheng'],
                               ['Eliot Bemis','Jesse Gan','Odie','Stouf','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Stouf','Swan','Carter Wang'],
                               ['Max Cohen','Victor Kao','Owen Marschall','Sam Pollack','Gabe Stump','Carter Thallon','Jeff Voss'],
                               ['Sam Alston','Eliot Bemis','Andrew Edelman','Stouf','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Victor Kao','Evan Ma','Owen Marschall','Ian McInerney','Odie','Sam Pollack','Gabe Stump'],
                               ['Eliot Bemis','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Ruozhou Ye','Geo Zheng'],
                               ['Sam Alston','Max Cohen','Jesse Gan','Victor Kao','Owen Marschall','Sam Pollack','Jeff Voss'],
                                ['Andrew Edelman','Jesse Gan','Juleon Robinson','Stouf','Swan','Towell','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Evan Ma','Owen Marschall','Odie','Sam Pollack','Gabe Stump'],
                               ['Sam Alston','Andrew Edelman','Odie','Carter Wang','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Andrew Edelman','Jesse Gan','Odie','Towell','Carter Wang','Ruozhou Ye','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Evan Ma','Sam Pollack','Gabe Stump','Jeff Voss'],
                               ['Sam Alston','Eliot Bemis','Andrew Edelman','Odie','Sam Pollack','Juleon Robinson','Ruozhou Ye'],
                               ['Andrew Edelman','Jesse Gan','Juleon Robinson','Stouf','Swan','Towell','Geo Zheng'],
                               ['Andrew Edelman','Odie','Juleon Robinson','Swan','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Carter Wang'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Odie','Gabe Stump','Carter Thallon'],
                               ['Andrew Edelman','Jesse Gan','Stouf','Jeff Voss','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Evan Ma','Owen Marschall','Odie','Sam Pollack','Jeff Voss']]}

In [33]:
hip_lineup_dict = {'opp_pt':['HIP.1','HIP.2','HIP.3','HIP.4','HIP.5',
                                'HIP.6','HIP.7','HIP.8','HIP.9','HIP.10',
                                'HIP.11','HIP.12','HIP.13','HIP.14','HIP.15',
                               'HIP.16','HIP.17','HIP.18','HIP.19'],
                      'lineup':[['Sam Alston','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye','Geo Zheng'],
                               ['Max Cohen','Victor Kao','Evan Ma','Ian McInerney','Gabe Stump','Carter Thallon','Jeff Voss'],
                               ['Andrew Edelman','Jesse Gan','Juleon Robinson','Stouf','Swan','Towell','Geo Zheng'],
                               ['Eliot Bemis','Victor Kao','Evan Ma','Owen Marschall','Odie','Sam Pollack','Jeff Voss'],
                               ['Sam Alston','Eliot Bemis','Andrew Edelman','Jeff Voss','Carter Wang','Ruozhou Ye','Geo Zheng'],
                               ['Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Gabe Stump','Carter Thallon'],
                               ['Sam Alston','Odie','Juleon Robinson','Stouf','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Sam Alston','Odie','Juleon Robinson','Stouf','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Sam Alston','Andrew Edelman','Stouf','Carter Wang','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Evan Ma','Owen Marschall','Ian McInerney','Sam Pollack','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                               ['Sam Alston','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Evan Ma','Gabe Stump','Carter Thallon','Jeff Voss'],
                               ['Jesse Gan','Odie','Sam Pollack','Juleon Robinson','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Sam Alston','Andrew Edelman','Josh Imhoff','Odie','Towell','Ruozhou Ye','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Ian McInerney','Gabe Stump','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Jesse Gan','Odie','Juleon Robinson','Carter Wang','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Evan Ma','Owen Marschall','Sam Pollack','Carter Thallon','Jeff Voss'],
                               ['Andrew Edelman','Josh Imhoff','Stouf','Towell','Ruozhou Ye','Zero','Geo Zheng']]}
pd.DataFrame(hip_lineup_dict)

,opp_pt,lineup
0,HIP.1,"[Sam Alston, Andrew Edelman, Jesse Gan, Josh I..."
1,HIP.2,"[Max Cohen, Victor Kao, Evan Ma, Ian McInerney..."
2,HIP.3,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S..."
3,HIP.4,"[Eliot Bemis, Victor Kao, Evan Ma, Owen Marsch..."
4,HIP.5,"[Sam Alston, Eliot Bemis, Andrew Edelman, Jeff..."
5,HIP.6,"[Max Cohen, Victor Kao, Owen Marschall, Odie, ..."
6,HIP.7,"[Sam Alston, Odie, Juleon Robinson, Stouf, Ruo..."
7,HIP.8,"[Sam Alston, Odie, Juleon Robinson, Stouf, Ruo..."
8,HIP.9,"[Sam Alston, Andrew Edelman, Stouf, Carter Wan..."
9,HIP.10,"[Eliot Bemis, Max Cohen, Evan Ma, Owen Marscha..."


In [34]:
nino_lineup_dict = {'opp_pt':['El Nino.1','El Nino.2','El Nino.3','El Nino.4','El Nino.5',
                                'El Nino.6','El Nino.7','El Nino.8','El Nino.9','El Nino.10',
                                'El Nino.11','El Nino.12','El Nino.13','El Nino.14','El Nino.15',
                               'El Nino.16','El Nino.17','El Nino.18','El Nino.19'],
                      'lineup':[['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Evan Ma','Sam Pollack','Gabe Stump','Jeff Voss'],
                               ['Max Cohen','Evan Ma','Owen Marschall','Ian McInerney','Odie','Carter Thallon','Jeff Voss'],
                               ['Andrew Edelman','Jesse Gan','Juleon Robinson','Swan','Towell','Ruozhou Ye','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Gabe Stump'],
                               ['Jesse Gan','Victor Kao','Evan Ma','Ian McInerney','Sam Pollack','Carter Thallon','Jeff Voss'],
                               ['Sam Alston','Alon Brown','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye'],
                               ['Alon Brown','Jon Kwan','Odie','Stouf','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Evan Ma','Ian McInerney','Gabe Stump','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                               ['Max Cohen','Evan Ma','Owen Marschall','Ian McInerney','Odie','Sam Pollack','Gabe Stump'],
                               ['Eliot Bemis','Max Cohen','Jesse Gan','Victor Kao','Owen Marschall','Carter Thallon','Jeff Voss'],
                               ['Andrew Edelman','Jesse Gan','Juleon Robinson','Stouf','Swan','Towell','Geo Zheng'],
                               ['Eliot Bemis','Victor Kao','Evan Ma','Owen Marschall','Ian McInerney','Sam Pollack','Carter Thallon'],
                               ['Alon Brown','Jon Kwan','Stouf','Towell','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Sam Alston','Alon Brown','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Evan Ma','Ian McInerney','Odie','Jeff Voss'],
                               ['Max Cohen','Evan Ma','Owen Marschall','Odie','Sam Pollack','Carter Thallon','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Carter Wang','Geo Zheng']]}
pd.DataFrame(nino_lineup_dict)

,opp_pt,lineup
0,El Nino.1,"[Sam Alston, Andrew Edelman, Josh Imhoff, Jule..."
1,El Nino.2,"[Eliot Bemis, Max Cohen, Victor Kao, Evan Ma, ..."
2,El Nino.3,"[Max Cohen, Evan Ma, Owen Marschall, Ian McIne..."
3,El Nino.4,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S..."
4,El Nino.5,"[Eliot Bemis, Max Cohen, Victor Kao, Owen Mars..."
5,El Nino.6,"[Jesse Gan, Victor Kao, Evan Ma, Ian McInerney..."
6,El Nino.7,"[Sam Alston, Alon Brown, Andrew Edelman, Jesse..."
7,El Nino.8,"[Alon Brown, Jon Kwan, Odie, Stouf, Ruozhou Ye..."
8,El Nino.9,"[Eliot Bemis, Max Cohen, Victor Kao, Evan Ma, ..."
9,El Nino.10,"[Sam Alston, Andrew Edelman, Josh Imhoff, Jule..."


In [35]:
citywide_lineup_dict = {'opp_pt':['Citywide.1','Citywide.2','Citywide.3','Citywide.4','Citywide.5',
                                'Citywide.6','Citywide.7','Citywide.8','Citywide.9','Citywide.10',
                                'Citywide.11','Citywide.12','Citywide.13','Citywide.14','Citywide.15',
                               'Citywide.16','Citywide.17','Citywide.18','Citywide.19','Citywide.20'],
                      'lineup':[['Sam Alston','Andrew Edelman','Jon Kwan','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                               ['Eliot Bemis','Max Cohen','Evan Ma','Owen Marschall','Ian McInerney','Sam Pollack','Jeff Voss'],
                               ['Andrew Edelman','Jesse Gan','Juleon Robinson','Stouf','Swan','Towell','Geo Zheng'],
                               ['Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Carter Thallon','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye','Geo Zheng'],
                               ['Eliot Bemis','Victor Kao','Evan Ma','Owen Marschall','Ian McInerney','Sam Pollack','Jeff Voss'],
                               ['Andrew Edelman','Jon Kwan','Stouf','Towell','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                               ['Victor Kao','Jon Kwan','Nadav Pearl','Juleon Robinson','Swan','Ruozhou Ye','Zero'],
                               ['Sam Alston','Andrew Edelman','Jesse Gan','Juleon Robinson','Stouf','Towell','Geo Zheng'],
                               ['Max Cohen','Victor Kao','Evan Ma','Odie','Sam Pollack','Carter Thallon','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Evan Ma','Owen Marschall','Ian McInerney','Odie','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Sam Pollack','Stouf','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Ian McInerney','Carter Thallon','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                               ['Jesse Gan','Jon Kwan','Nadav Pearl','Juleon Robinson','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Victor Kao','Evan Ma','Owen Marschall','Ian McInerney','Odie','Sam Pollack','Jeff Voss'],
                               ['Sam Alston','Jesse Gan','Josh Imhoff','Victor Kao','Nadav Pearl','Carter Wang','Geo Zheng'],
                               ['Jesse Gan','Juleon Robinson','Stouf','Swan','Towell','Ruozhou Ye','Geo Zheng']]}
pd.DataFrame(citywide_lineup_dict)

,opp_pt,lineup
0,Citywide.1,"[Sam Alston, Andrew Edelman, Jon Kwan, Juleon ..."
1,Citywide.2,"[Eliot Bemis, Max Cohen, Evan Ma, Owen Marscha..."
2,Citywide.3,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S..."
3,Citywide.4,"[Max Cohen, Victor Kao, Owen Marschall, Odie, ..."
4,Citywide.5,"[Sam Alston, Andrew Edelman, Jesse Gan, Josh I..."
5,Citywide.6,"[Eliot Bemis, Victor Kao, Evan Ma, Owen Marsch..."
6,Citywide.7,"[Andrew Edelman, Jon Kwan, Stouf, Towell, Ruoz..."
7,Citywide.8,"[Sam Alston, Andrew Edelman, Josh Imhoff, Jule..."
8,Citywide.9,"[Victor Kao, Jon Kwan, Nadav Pearl, Juleon Rob..."
9,Citywide.10,"[Sam Alston, Andrew Edelman, Jesse Gan, Juleon..."


In [36]:
floodwall_lineup_dict = {'opp_pt':['Floodwall.1','Floodwall.2','Floodwall.3','Floodwall.4','Floodwall.5',
                                'Floodwall.6','Floodwall.7','Floodwall.8','Floodwall.9','Floodwall.10',
                                'Floodwall.11','Floodwall.12','Floodwall.13','Floodwall.14','Floodwall.15',
                               'Floodwall.16','Floodwall.17','Floodwall.18','Floodwall.19','Floodwall.20',
                                  'Floodwall.21'],
                      'lineup':[['Max Cohen','Victor Kao','Evan Ma','Odie','Sam Pollack','Gabe Stump','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                               ['Alon Brown','Jon Kwan','Stouf','Towell','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Sam Alston','Alon Brown','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye'],
                               ['Eliot Bemis','Max Cohen','Evan Ma','Owen Marschall','Ian McInerney','Odie','Carter Thallon'],
                               ['Andrew Edelman','Jesse Gan','Juleon Robinson','Stouf','Swan','Towell','Geo Zheng'],
                               ['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Sam Pollack','Gabe Stump','Jeff Voss'],
                               ['Alon Brown','Jon Kwan','Nadav Pearl','Juleon Robinson','Swan','Zero','Geo Zheng'],
                               ['Sam Alston','Alon Brown','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye'],
                               ['Victor Kao','Evan Ma','Owen Marschall','Ian McInerney','Gabe Stump','Carter Thallon','Jeff Voss'],
                               ['Sam Alston','Alon Brown','Jon Kwan','Stouf','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Evan Ma','Ian McInerney','Odie','Sam Pollack'],
                               ['Max Cohen','Evan Ma','Owen Marschall','Sam Pollack','Gabe Stump','Carter Thallon','Jeff Voss'],
                               ['Andrew Edelman','Jesse Gan','Juleon Robinson','Stouf','Swan','Towell','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Evan Ma','Owen Marschall','Ian McInerney','Odie','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                                ['Max Cohen','Victor Kao','Owen Marschall','Odie','Sam Pollack','Gabe Stump','Carter Thallon'],
                               ['Sam Alston','Alon Brown','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye'],
                                ['Alon Brown','Jon Kwan','Nadav Pearl','Juleon Robinson','Swan','Zero','Geo Zheng'],
                               ['Eliot Bemis','Victor Kao','Evan Ma','Owen Marschall','Ian McInerney','Gabe Stump','Jeff Voss']]}
pd.DataFrame(floodwall_lineup_dict)

,opp_pt,lineup
0,Floodwall.1,"[Max Cohen, Victor Kao, Evan Ma, Odie, Sam Pol..."
1,Floodwall.2,"[Sam Alston, Andrew Edelman, Josh Imhoff, Jule..."
2,Floodwall.3,"[Alon Brown, Jon Kwan, Stouf, Towell, Ruozhou ..."
3,Floodwall.4,"[Sam Alston, Alon Brown, Andrew Edelman, Jesse..."
4,Floodwall.5,"[Eliot Bemis, Max Cohen, Evan Ma, Owen Marscha..."
5,Floodwall.6,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S..."
6,Floodwall.7,"[Sam Alston, Andrew Edelman, Josh Imhoff, Jule..."
7,Floodwall.8,"[Eliot Bemis, Max Cohen, Victor Kao, Owen Mars..."
8,Floodwall.9,"[Alon Brown, Jon Kwan, Nadav Pearl, Juleon Rob..."
9,Floodwall.10,"[Sam Alston, Alon Brown, Andrew Edelman, Jesse..."


In [37]:
lostboys_lineup_dict = {'opp_pt':['Lost  Boys.1','Lost  Boys.2','Lost  Boys.3','Lost  Boys.4','Lost  Boys.5','Lost  Boys.6',
                                  'Lost  Boys.7','Lost  Boys.8','Lost  Boys.9','Lost  Boys.10','Lost  Boys.11',
                                  'Lost  Boys.12','Lost  Boys.13','Lost  Boys.14'],
                      'lineup':[['Eliot Bemis','Max Cohen','Victor Kao','Evan Ma','Odie','Sam Pollack','Carter Thallon'],
                               ['Sam Alston','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye','Geo Zheng'],
                               ['Andrew Edelman','Jesse Gan','Juleon Robinson','Stouf','Swan','Towell','Geo Zheng'],
                               ['Max Cohen','Evan Ma','Owen Marschall','Ian McInerney','Odie','Sam Pollack','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye','Geo Zheng'],
                               ['Jon Kwan','Nadav Pearl','Juleon Robinson','Jeff Voss','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Andrew Edelman','Jesse Gan','Jon Kwan','Stouf','Ruozhou Ye','Zero','Geo Zheng'],
                               ['Andrew Edelman','Jesse Gan','Juleon Robinson','Stouf','Swan','Towell','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Owen Marschall','Ian McInerney','Carter Thallon','Jeff Voss'],
                               ['Sam Alston','Andrew Edelman','Josh Imhoff','Juleon Robinson','Towell','Carter Wang','Ruozhou Ye'],
                               ['Eliot Bemis','Victor Kao','Evan Ma','Owen Marschall','Ian McInerney','Odie','Sam Pollack'],
                               ['Sam Alston','Andrew Edelman','Jesse Gan','Josh Imhoff','Carter Wang','Ruozhou Ye','Geo Zheng'],
                               ['Eliot Bemis','Max Cohen','Victor Kao','Evan Ma','Sam Pollack','Carter Thallon','Jeff Voss'],
                               ['Eliot Bemis','Jesse Gan','Jon Kwan','Towell','Ruozhou Ye','Zero','Geo Zheng']]}
pd.DataFrame(lostboys_lineup_dict)

,opp_pt,lineup
0,Lost Boys.1,"[Eliot Bemis, Max Cohen, Victor Kao, Evan Ma, ..."
1,Lost Boys.2,"[Sam Alston, Andrew Edelman, Jesse Gan, Josh I..."
2,Lost Boys.3,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S..."
3,Lost Boys.4,"[Max Cohen, Evan Ma, Owen Marschall, Ian McIne..."
4,Lost Boys.5,"[Sam Alston, Andrew Edelman, Jesse Gan, Josh I..."
5,Lost Boys.6,"[Jon Kwan, Nadav Pearl, Juleon Robinson, Jeff ..."
6,Lost Boys.7,"[Andrew Edelman, Jesse Gan, Jon Kwan, Stouf, R..."
7,Lost Boys.8,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S..."
8,Lost Boys.9,"[Eliot Bemis, Max Cohen, Victor Kao, Owen Mars..."
9,Lost Boys.10,"[Sam Alston, Andrew Edelman, Josh Imhoff, Jule..."


In [38]:
#create a list of all dictionaries of lineups
dicts = [lantern_lineup_dict,wrench_lineup_dict,shade_lineup_dict,vault_lineup_dict,circus_lineup_dict,
        hip_lineup_dict,nino_lineup_dict,citywide_lineup_dict,floodwall_lineup_dict,lostboys_lineup_dict]

In [39]:
#turn all game dictionaries into one lineup dataframe
lineups = pd.DataFrame(columns=['opp_pt','lineup'])
for d in dicts:
    lineups = lineups.append(pd.DataFrame(d))
lineups

,opp_pt,lineup
0,Lantern.1,[]
1,Lantern.2,"[Max Cohen, Jesse Gan, Owen Marschall, Odie, S..."
2,Lantern.3,"[Alon Brown, Andrew Edelman, Jon Kwan, Evan Ma..."
3,Lantern.4,"[Max Cohen, Owen Marschall, Odie, Sam Pollack,..."
4,Lantern.5,"[Alon Brown, Andrew Edelman, Sam Feder, Jesse ..."
...,...,...
9,Lost Boys.10,"[Sam Alston, Andrew Edelman, Josh Imhoff, Jule..."
10,Lost Boys.11,"[Eliot Bemis, Victor Kao, Evan Ma, Owen Marsch..."
11,Lost Boys.12,"[Sam Alston, Andrew Edelman, Jesse Gan, Josh I..."
12,Lost Boys.13,"[Eliot Bemis, Max Cohen, Victor Kao, Evan Ma, ..."


In [40]:
#merge lineup dfs with lineups on the opp_pt column
lineup_df = lineup_df.merge(lineups,how='outer')

In [41]:
#edit the opp_pt column so that it better fits the synatx of the possession index column
lineup_df['opp_pt'].replace('.',' | ')

0        Lost  Boys.1
1        Lost  Boys.1
2        Lost  Boys.1
3        Lost  Boys.1
4        Lost  Boys.1
            ...      
1198     Floodwall.15
1199     Floodwall.17
1200     Floodwall.20
1201     Lost  Boys.8
1202    Lost  Boys.10
Name: opp_pt, Length: 1203, dtype: object

In [42]:
#complete edits to make the opp_pt column the same as the index column
#for all defensive points that never have a possession, we're gonna index them at '.0'
lineup_df['possession_index'] = lineup_df['possession_index'].fillna(lineup_df['opp_pt']
                                                                     .str.replace('.',' | ') + '.0')

In [43]:
#drop now unneccessary columns from the lineup df
lineup_df = lineup_df.drop(columns=['opponent','Point','Possession','opp_pt'])

In [44]:
#make sure the lineup df only has unique point values
lineup_df = lineup_df.drop_duplicates('possession_index')

In [53]:
#create a lineup index that will exist for calling lineups from the other dataframes
lineup_df['lineup_id'] = lineup_df.index
lineup_df

,possession_index,lineup,lineup_id
0,Lost Boys | 1.1,"[Eliot Bemis, Max Cohen, Victor Kao, Evan Ma, ...",0
7,Lost Boys | 2.1,"[Sam Alston, Andrew Edelman, Jesse Gan, Josh I...",7
9,Lost Boys | 3.1,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S...",9
14,Lost Boys | 3.2,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S...",14
17,Lost Boys | 4.1,"[Max Cohen, Evan Ma, Owen Marschall, Ian McIne...",17
...,...,...,...
1198,Floodwall | 15.0,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S...",1198
1199,Floodwall | 17.0,"[Sam Alston, Andrew Edelman, Josh Imhoff, Jule...",1199
1200,Floodwall | 20.0,"[Alon Brown, Jon Kwan, Nadav Pearl, Juleon Rob...",1200
1201,Lost Boys | 8.0,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S...",1201


In [55]:
#explode the lineup lists into individual names
lineup_df = lineup_df.explode('lineup')

In [46]:
#merge the lineup df back with the full_poss_df, making sure to outer merge so that the new possession are included
full_poss_df = full_poss_df.merge(lineup_df,how='outer')

In [47]:
#fill the scored and offense columns with 0s for all of the new defensive only possessions
full_poss_df[['Scored?','Started point on offense?']] = full_poss_df[['Scored?',
                                                                      'Started point on offense?']].fillna(float(0))

In [48]:
#create a dataframe with only unique possession indices
unique_poss_df = full_poss_df.drop_duplicates(['possession_index','Passes'])

In [52]:
unique_poss_df

,Created,Point,Possession,Thrower,Receiver,Turnover?,Thrower error?,Receiver error?,Throw to endzone?,Assist?,...,Passes,Secondary assist,Assist,Goal,Thrower error,Receiver error,Stalled out,possession_index,lineup,lineup_id
0,2020-02-26 22:56:29,1.0,1.0,Evan Ma,Max Cohen,0.0,0.0,0.0,0.0,0.0,...,7.0,Carter Thallon,Odie,Eliot Bemis,NaN,NaN,NaN,Lost Boys | 1.1,"[Eliot Bemis, Max Cohen, Victor Kao, Evan Ma, ...",0
7,2020-02-26 22:58:06,2.0,1.0,Andrew Edelman,Josh Imhoff,0.0,0.0,0.0,0.0,0.0,...,2.0,Andrew Edelman,Josh Imhoff,Sam Alston,NaN,NaN,NaN,Lost Boys | 2.1,"[Sam Alston, Andrew Edelman, Jesse Gan, Josh I...",7
9,2020-02-26 22:59:21,3.0,1.0,Andrew Edelman,Geo Zheng,0.0,0.0,0.0,0.0,0.0,...,5.0,NaN,NaN,NaN,Stouf,NaN,NaN,Lost Boys | 3.1,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S...",9
14,2020-02-26 23:00:22,3.0,2.0,Andrew Edelman,Towell,0.0,0.0,0.0,0.0,0.0,...,3.0,NaN,NaN,NaN,Andrew Edelman,NaN,NaN,Lost Boys | 3.2,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S...",14
17,2020-02-26 23:08:14,4.0,1.0,Evan Ma,Max Cohen,0.0,0.0,0.0,0.0,0.0,...,6.0,NaN,NaN,NaN,Evan Ma,NaN,NaN,Lost Boys | 4.1,"[Max Cohen, Evan Ma, Owen Marschall, Ian McIne...",17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Floodwall | 15.0,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S...",1198
1199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Floodwall | 17.0,"[Sam Alston, Andrew Edelman, Josh Imhoff, Jule...",1199
1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Floodwall | 20.0,"[Alon Brown, Jon Kwan, Nadav Pearl, Juleon Rob...",1200
1201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lost Boys | 8.0,"[Andrew Edelman, Jesse Gan, Juleon Robinson, S...",1201


In [50]:
#check that all names are spelled correctly
names = []
for i in unique_poss_df['lineup']:
    for j in i:
        names.append(j)
len(np.unique(names))

28

In [51]:
full_poss_df[full_poss_df['lineup'].str.contains('Max Cohen',regex=False)]

,Created,Point,Possession,Thrower,Receiver,Turnover?,Thrower error?,Receiver error?,Throw to endzone?,Assist?,...,Passes,Secondary assist,Assist,Goal,Thrower error,Receiver error,Stalled out,possession_index,lineup,lineup_id
0,2020-02-26 22:56:29,1.0,1.0,Evan Ma,Max Cohen,0.0,0.0,0.0,0.0,0.0,...,7.0,Carter Thallon,Odie,Eliot Bemis,NaN,NaN,NaN,Lost Boys | 1.1,"[Eliot Bemis, Max Cohen, Victor Kao, Evan Ma, ...",0
1,2020-02-26 22:56:39,1.0,1.0,Max Cohen,Victor Kao,0.0,0.0,0.0,0.0,0.0,...,7.0,Carter Thallon,Odie,Eliot Bemis,NaN,NaN,NaN,Lost Boys | 1.1,"[Eliot Bemis, Max Cohen, Victor Kao, Evan Ma, ...",0
2,2020-02-26 22:56:46,1.0,1.0,Victor Kao,Carter Thallon,0.0,0.0,0.0,0.0,0.0,...,7.0,Carter Thallon,Odie,Eliot Bemis,NaN,NaN,NaN,Lost Boys | 1.1,"[Eliot Bemis, Max Cohen, Victor Kao, Evan Ma, ...",0
3,2020-02-26 22:56:49,1.0,1.0,Carter Thallon,Odie,0.0,0.0,0.0,0.0,0.0,...,7.0,Carter Thallon,Odie,Eliot Bemis,NaN,NaN,NaN,Lost Boys | 1.1,"[Eliot Bemis, Max Cohen, Victor Kao, Evan Ma, ...",0
4,2020-02-26 22:56:55,1.0,1.0,Odie,Carter Thallon,0.0,0.0,0.0,0.0,0.0,...,7.0,Carter Thallon,Odie,Eliot Bemis,NaN,NaN,NaN,Lost Boys | 1.1,"[Eliot Bemis, Max Cohen, Victor Kao, Evan Ma, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,2020-02-18 19:01:10,14.0,1.0,Owen Marschall,Max Cohen,0.0,0.0,0.0,0.0,0.0,...,5.0,Jeff Voss,Carter Thallon,Odie,NaN,NaN,NaN,Lantern | 14.1,"[Max Cohen, Owen Marschall, Odie, Sam Pollack,...",1158
1159,2020-02-18 19:01:19,14.0,1.0,Max Cohen,Sam Pollack,0.0,0.0,0.0,0.0,0.0,...,5.0,Jeff Voss,Carter Thallon,Odie,NaN,NaN,NaN,Lantern | 14.1,"[Max Cohen, Owen Marschall, Odie, Sam Pollack,...",1158
1160,2020-02-18 19:01:25,14.0,1.0,Sam Pollack,Jeff Voss,0.0,0.0,0.0,0.0,0.0,...,5.0,Jeff Voss,Carter Thallon,Odie,NaN,NaN,NaN,Lantern | 14.1,"[Max Cohen, Owen Marschall, Odie, Sam Pollack,...",1158
1161,2020-02-18 19:01:29,14.0,1.0,Jeff Voss,Carter Thallon,0.0,0.0,0.0,0.0,0.0,...,5.0,Jeff Voss,Carter Thallon,Odie,NaN,NaN,NaN,Lantern | 14.1,"[Max Cohen, Owen Marschall, Odie, Sam Pollack,...",1158


In [ ]:
#classify passes as centering passes based on being first point of an offensive point, not starting near the brick mark
#look at everyone's completion percentage and stats within the endzone set

### chart the number of passes per possession and scoring rate

In [ ]:
#create passes per possession bar chart

pass_count_list = sorted(unique_poss_df['Passes'])
c = Counter(pass_count_list)
plt.bar(c.keys(),c.values(),label='Total Possessions')

score_unique_poss_df = unique_poss_df[unique_poss_df['Scored?'] == 1]
score_pass_count_list = sorted(score_unique_poss_df['Passes'])
d = Counter(score_pass_count_list)
plt.bar(d.keys(),d.values(),label='Scoring Possessions')

plt.xticks(np.arange(1,max(sorted(np.unique(pass_count_list)))+1))
plt.yticks(np.arange(max(c.values())+1,step=2))
plt.xlabel('Passes')
plt.ylabel('# Possessions')
plt.title('Passes per Possession')
plt.legend()
plt.savefig('figs/passes_per_possession.png', dpi=300)
plt.show()

In [ ]:
most_passes = full_poss_df['Passes'].max()
full_poss_df[full_poss_df['Passes'] == most_passes]

### chart all throws on the field

In [ ]:
turnover_df = full_poss_df[full_poss_df['Thrower error?'] == 1]

In [ ]:
assist_df = full_poss_df[full_poss_df['Assist?'] == 1]

In [ ]:
just_throw_df = full_poss_df[(full_poss_df['Assist?'] == 0) & (full_poss_df['Thrower error?'] == 0)]

In [ ]:
#scatter plot of all throwing positions
fig = plt.figure(figsize=(4/1.2,11/1.2))

#all normal throws
plt.scatter(just_throw_df['Start X (0 -> 1 = left sideline -> right sideline)'],
           1-just_throw_df['Start Y (0 -> 1 = back of opponent endzone -> back of own endzone)'],
           alpha=.5)

#assists
plt.scatter(assist_df['Start X (0 -> 1 = left sideline -> right sideline)'],
           1-assist_df['Start Y (0 -> 1 = back of opponent endzone -> back of own endzone)'],
           alpha=1,
            c='g',
            label='Assist')

#turnovers
plt.scatter(turnover_df['Start X (0 -> 1 = left sideline -> right sideline)'],
           1-turnover_df['Start Y (0 -> 1 = back of opponent endzone -> back of own endzone)'],
           alpha=1,
            marker='x',
            c='r',
            label='Turnover')

#attacking endzone line
x1 = [0,1]
y1 = [.8181818,.8181818]
plt.plot(x1,y1)

#defending endzone line
x2 = [0,1]
y2 = [.1818181,.1818181]
plt.plot(x2,y2)

#backhand sideline
x3 = [0,0]
y3 = [0,1]
plt.plot(x3,y3)

#flick sideline
x4 = [1,1]
y4 = [0,1]
plt.plot(x4,y4)

#attacking backline
x5 = [0,1]
y5 = [1,1]
plt.plot(x5,y5)

#defending backline
x6 = [0,1]
y6 = [0,0]
plt.plot(x6,y6)

xtick = .25/2
ytick = .181818/2

plt.xticks(ticks=[xtick,xtick*2,xtick*3,xtick*4,xtick*5,xtick*6,xtick*7],
          labels=[5,10,15,20,15,10,5])
plt.yticks(ticks=[ytick*2,ytick*3,ytick*4,ytick*5,ytick*6,ytick*7,ytick*8,ytick*9],
          labels=[0,10,20,30,40,50,60,'Goal'])
#plt.xlabel()
#plt.ylabel()
plt.legend(loc='upper right')
plt.title('All Throw Starting Positions')

plt.show()
fig.savefig('figs/throw_start_scatter.png',dpi=300,bbox_inches='tight')

### Scoring percent by vertical position on field

In [ ]:
#create chart that shows scoring probbality of scoring based on distance from the attacking endzone

In [ ]:
#create bins every two yards
bin_list = []
yardage = 3
for x in range(-3,109,yardage):
    bin_list.append(x + yardage)
    
bins = np.array(bin_list)

In [ ]:
#change bins to 0->1 range
bins = [x/110 for x in bins]
bins

In [ ]:
full_poss_df['y_bucket'] = pd.cut(full_poss_df['Start Y (0 -> 1 = back of opponent endzone -> back of own endzone)'],
                              bins)
full_poss_df['y_bucket']

In [ ]:
all_hist = np.histogram(1-full_poss_df['Start Y (0 -> 1 = back of opponent endzone -> back of own endzone)'],bins)
len(all_hist[0])

In [ ]:
scored_df = full_poss_df[full_poss_df['Scored?'] == 1]

In [ ]:
scored_hist = np.histogram(1-scored_df['Start Y (0 -> 1 = back of opponent endzone -> back of own endzone)'],bins)
len(scored_hist[0])

In [ ]:
all_y = all_hist[0]
scored_y = scored_hist[0]
bins_data = bins[:-1]
d = {'all_y':all_y, 'scored_y':scored_y, 'bins':bins_data}
y_score_df = pd.DataFrame(data = d)

In [ ]:
bins[:-1]

In [ ]:
y_score_df['conversion_rate'] = y_score_df['scored_y'] / y_score_df['all_y']

In [ ]:
y_score_df['yards'] = y_score_df.index * yardage
y_score_df['yards']

In [ ]:
y_score_df = y_score_df.dropna()
y_score_df = y_score_df[y_score_df['all_y'] > 10]
y_score_df

In [ ]:
#by vertical position on the field
#chance of scoring by field position
fig = plt.figure(figsize=(9,4))

x = y_score_df['yards']+(.5*yardage)#<-center the point in the middle of the bin
y = y_score_df['conversion_rate']
#s = y_score_df['all_y']
#plt.scatter(x,y,s=s)

z = np.polyfit(x, y, 3)
p = np.poly1d(z)
plt.plot(x,p(x),"g")

xtick = 10
ytick = .1

plt.xticks(ticks=[xtick*9,xtick*8,xtick*7,xtick*6,xtick*5,xtick*4,xtick*3,xtick*2,xtick*1,xtick*0],
          labels=[90,80,'Own\nEndzone',60,50,40,30,20,10,'Scoring\nEndzone'][::-1])
plt.yticks(ticks=[ytick,ytick*2,ytick*3,ytick*4,ytick*5,ytick*6,ytick*7,ytick*8,ytick*9,ytick*10])
plt.xlabel('Yards from scoring')
plt.ylabel('Chance of scoring')
#plt.legend(loc='upper right')
plt.title('Chance of scoring by field position')

plt.grid(which='major', axis='y' ,linestyle='-.', linewidth='.2', color='black')
plt.show()
fig.savefig('figs/chance_score_field_position.png',dpi=300, 
            bbox_inches= 'tight')

### Scoring percent by horizontal position on field

In [ ]:
#create chart that shows scoring probbality of scoring based on horizontal position on field

In [ ]:
#create bins every two yards
ho_bin_list = [0]
ho_yardage = 2
for x in range(0,40,ho_yardage):
    ho_bin_list.append(x + ho_yardage)
    
ho_bins = np.array(ho_bin_list)

In [ ]:
#change bins to 0->1 range
ho_bins = [x/40 for x in ho_bins]
ho_bins

In [ ]:
full_poss_df['x_bucket'] = pd.cut(full_poss_df['Start X (0 -> 1 = left sideline -> right sideline)'],
                              ho_bins)
full_poss_df['x_bucket']

In [ ]:
ho_all_hist = np.histogram(full_poss_df['Start X (0 -> 1 = left sideline -> right sideline)'],ho_bins)
len(ho_all_hist[0])

In [ ]:
scored_df

In [ ]:
ho_scored_hist = np.histogram(scored_df['Start X (0 -> 1 = left sideline -> right sideline)'],ho_bins)
len(ho_scored_hist[0])

In [ ]:
all_x = ho_all_hist[0]
scored_x = ho_scored_hist[0]
ho_bins_data = ho_bins[:-1]
ho_d = {'all_x':all_x, 'scored_x':scored_x, 'bins':ho_bins_data}
x_score_df = pd.DataFrame(data = ho_d)

In [ ]:
ho_bins[:-1]

In [ ]:
x_score_df['conversion_rate'] = x_score_df['scored_x'] / x_score_df['all_x']

In [ ]:
x_score_df['yards'] = x_score_df.index * ho_yardage
x_score_df['yards']

In [ ]:
x_score_df = x_score_df.dropna()
x_score_df = x_score_df[x_score_df['all_x'] > 10]
x_score_df

In [ ]:
#by horizontal position on the field
#chance of scoring by field position
fig = plt.figure(figsize=(4,4))

x = x_score_df['yards']+(.5*ho_yardage)#<-center the point in the middle of the bin
y = x_score_df['conversion_rate']
s = x_score_df['all_x']
plt.scatter(x,y,s)

z = np.polyfit(x, y, 6)
p = np.poly1d(z)
plt.plot(x,p(x),"g")

xtick = 5
ytick = .1

plt.xticks(ticks=[xtick*8,xtick*7,xtick*6,xtick*5,xtick*4,xtick*3,xtick*2,xtick*1,xtick*0],
          labels=[0,5,10,15,20,15,10,5,0])
plt.yticks(ticks=[ytick,ytick*2,ytick*3,ytick*4,ytick*5,ytick*6,ytick*7,ytick*8,ytick*9,ytick*10])
plt.xlabel('Distance from sideline')
plt.ylabel('Chance of scoring')
#plt.legend(loc='upper right')
plt.title('Chance of scoring by horizontal field position')

plt.grid(which='major', axis='y' ,linestyle='-.', linewidth='.2', color='black')
plt.show()
fig.savefig('figs/chance_score_HORIZONTAL_field_position.png',dpi=300, 
            bbox_inches= 'tight')

In [ ]:
full_poss_df

In [ ]:
full_poss_df[(full_poss_df['Thrower error?'] == 1) & (full_poss_df['Receiver error?'] == 1)]

In [ ]:
full_poss_df['Turnover?'].sum()

In [ ]:
full_poss_df['Thrower error?'].sum() + full_poss_df['Receiver error?'].sum()

### everything below here is just leftover remnants of some other ideas

### ADD ALL GAMES DF

games_file = 'Blueprint 2019 Games.csv'
games_df = pd.read_csv(games_file)
games_df

### DETERMINE D TO O RATIOS 

In [ ]:
#this was a brief attempt to figure out how to balance the stats of defensive players

#determine the qualifier between o points and d points
#how many more yards are gained on o points than d points
#can use this as some sort of measuring stick to qualify players offensive performance

#we can combine the passes and possessions dfs 
passes_df.columns.intersection(poss_df.columns)

full_poss_df.columns

#how many more yards are GAINED on o points than d points
o_dist_gain = full_poss_df[['Forward distance (m)']].where(full_poss_df['Started point on offense?'] == 1).sum()[0]
d_dist_gain = full_poss_df[['Forward distance (m)']].where(full_poss_df['Started point on offense?'] == 0).sum()[0]

#how many more yards are TOTAL THROWN on o points than d points
o_dist_total = full_poss_df[['Distance (m)']].where(full_poss_df['Started point on offense?'] == 1).sum()[0]
d_dist_total = full_poss_df[['Distance (m)']].where(full_poss_df['Started point on offense?'] == 0).sum()[0]

mult_d_total = ((o_dist_total + d_dist_total) / 2) / d_dist_total
mult_o_total = ((o_dist_total + d_dist_total) / 2) / o_dist_total

mult_d_gain = ((o_dist_gain / d_dist_gain) / 2) / d_dist_gain
mult_o_gain = ((o_dist_gain / d_dist_gain) / 2) / o_dist_gain

#find o to d pp ratio
player_stats_df['o_d_pp_ratio'] = player_stats_df['Offense points played'] / player_stats_df['Points played total']

o_pp_ratio = player_stats_df.loc[0,'o_d_pp_ratio']

player_stats_df.loc[0,'Total completed throw distance (m)']

o_pp_ratio * abs(1-mult_d_total)